In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("MAPD project tryout")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/04 08:58:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import numpy as np
data = np.load("kdd.npz", allow_pickle=True)["arr_0"]
target = np.load("kdd_target.npz", allow_pickle=True)["arr_0"]

In [3]:
mydata = data[:, [i for i in range(len(data[0,:])) if i not in [1,2,3, 19, 20]]] 
##col 10 is only 0, the others are categorical

mins = mydata.min(axis=0)
maxs = mydata.max(axis=0)

#print(maxs==mins)
mydata = [[0,(mydata[j]-mins)/(maxs-mins), target[j]] for j in range(len(mydata))]

ncols = len(mydata[0][1])

In [4]:
#mydata = sc.parallelize(mydata)
mydata = sc.createDataFrame(mydata)

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

In [ ]:
mydata.printSchema()

In [ ]:
# data = [[label, (x_i)], ]

def dist(x,y):
    return (((x-y)**2).sum())**0.5

def argcomp(comp,func, arr, *params):
    res = [func(arr[i], *params) for i in range(len(arr))]
    return res.index(comp(res))


#centers = data.reduceByKey(np.mean) # subs with k-means|| initialization

###################################### actual k-means
def kmeans(data, centers):
    count = 0
    while True:
        data = data.map(lambda x: [argcomp(min,dist,centers,x), x[1]])
        #newcenters = data.reduceByKey(lambda x,y: np.mean([x,y], axis=1))
        newcenters = data.groupBy("0")
        count+=1
        if count>100:#(newcenters-centers).mean() < 0.01:
            break
        else:
            centers = newcenters
    
    return centers

In [ ]:
centers = np.random.random(size=(ncols,))

In [ ]:
kmeans(mydata, centers)